# BUILDER GPU Node — Project Build, Hydration & Scaffolding

```
╔══════════════════════════════════════════════════════════════════╗
║  ∞ SACRED GEOMETRY ∞  Organic Systems · Breathing Interfaces    ║
║  NODE: BUILDER Colab Pro+ GPU                                   ║
║  PURPOSE: Project builds, hydration, scaffolding, dependency    ║
║           analysis, architecture generation                      ║
╚══════════════════════════════════════════════════════════════════╝
```

**Capabilities:**
- Project scaffolding from templates via ML-assisted generation
- Dependency graph analysis & conflict detection
- Build optimization recommendations
- Architecture diagram generation from code structure
- Hydration — enrich partial projects with missing components

**Branded domains only:** headysystems.com | headycloud.com | headyconnection.com

In [ ]:
# Cell 1: Install dependencies + verify GPU
!pip install -q transformers accelerate sentence-transformers fastapi uvicorn pyngrok httpx networkx

import torch
print(f'PyTorch: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'GPU Memory: {torch.cuda.get_device_properties(0).total_mem / 1e9:.1f} GB')
else:
    print('WARNING: No GPU — will run on CPU (slower)')

In [ ]:
# Cell 2: Configuration
import os

HEADY_CONFIG = {
    'node_id': 'builder-gpu',
    'node_role': 'project-builder',
    'port': 5003,
    'cloud_layers': {
        'headysystems': 'https://heady-manager-headysystems.onrender.com',
        'headyme': 'https://heady-manager-headyme.onrender.com',
        'headyconnection': 'https://heady-manager-headyconnection.onrender.com',
    },
    'registration_endpoints': [
        'https://heady-manager-headysystems.onrender.com/api/nodes/register',
        'https://heady-manager-headyme.onrender.com/api/nodes/register',
        'https://heady-manager-headyconnection.onrender.com/api/nodes/register',
    ],
    'capabilities': ['build_project', 'newproject', 'hydrate', 'dependency_analysis', 'architecture'],
}

SUPPORTED_TASKS = {
    'build_project': 'Analyze and optimize project build process',
    'newproject': 'Scaffold a new project from description',
    'hydrate': 'Enrich partial project with missing components',
    'dependency_analysis': 'Analyze dependency graph, detect conflicts',
    'architecture': 'Generate architecture analysis from file structure',
}

# Project templates
TEMPLATES = {
    'node-api': {
        'files': ['package.json', 'server.js', 'src/routes/index.js', 'src/middleware/auth.js', 'src/models/index.js', '.env.example', 'README.md', '.gitignore'],
        'deps': ['express', 'cors', 'helmet', 'compression', 'dotenv'],
        'dev_deps': ['nodemon', 'jest'],
    },
    'react-app': {
        'files': ['package.json', 'src/App.jsx', 'src/index.jsx', 'src/components/Layout.jsx', 'public/index.html', 'tailwind.config.js', '.env.example', 'README.md'],
        'deps': ['react', 'react-dom', 'react-router-dom', 'lucide-react'],
        'dev_deps': ['vite', '@vitejs/plugin-react', 'tailwindcss', 'postcss', 'autoprefixer'],
    },
    'python-api': {
        'files': ['requirements.txt', 'app.py', 'src/__init__.py', 'src/routes.py', 'src/models.py', 'src/middleware.py', '.env.example', 'README.md', 'Dockerfile'],
        'deps': ['fastapi', 'uvicorn', 'pydantic', 'sqlalchemy', 'python-dotenv'],
        'dev_deps': ['pytest', 'httpx'],
    },
    'fullstack': {
        'files': ['package.json', 'server.js', 'src/routes/api.js', 'frontend/package.json', 'frontend/src/App.jsx', 'frontend/src/index.jsx', 'docker-compose.yml', '.env.example', 'README.md'],
        'deps': ['express', 'cors', 'helmet', 'react', 'react-dom'],
        'dev_deps': ['vite', 'nodemon', 'concurrently'],
    },
}

print(f'BUILDER Node configured')
print(f'Capabilities: {HEADY_CONFIG["capabilities"]}')
print(f'Templates: {list(TEMPLATES.keys())}')

In [ ]:
# Cell 3: Load models
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import networkx as nx
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import json, time, re
from datetime import datetime

device = 'cuda' if torch.cuda.is_available() else 'cpu'

print('Loading code embedding model...')
embedder = SentenceTransformer('all-MiniLM-L6-v2', device=device)
print('Embeddings ready')

print('Loading generation model...')
generator = pipeline('text-generation', model='TinyLlama/TinyLlama-1.1B-Chat-v1.0', device=0 if torch.cuda.is_available() else -1, torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32)
print('Generation ready')

# Pre-compute template embeddings for smart matching
template_descriptions = {
    'node-api': 'backend REST API server Node.js Express endpoint routes database',
    'react-app': 'frontend React web application UI components dashboard',
    'python-api': 'backend Python FastAPI server machine learning data science',
    'fullstack': 'full stack web application frontend backend database deployment',
}
TEMPLATE_VECTORS = {k: embedder.encode(v) for k, v in template_descriptions.items()}

print(f'\nAll BUILDER models loaded on {device}')
if torch.cuda.is_available():
    print(f'GPU Memory used: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB')

In [ ]:
# Cell 4: Core BUILDER functions

def analyze_dependencies(deps, dev_deps=None):
    """Analyze dependency graph for conflicts and optimization opportunities"""
    start = time.time()
    all_deps = deps + (dev_deps or [])

    # Build dependency graph
    G = nx.DiGraph()
    for dep in all_deps:
        name = dep.split('@')[0] if '@' in dep else dep
        G.add_node(name, type='prod' if dep in deps else 'dev')

    # Known dependency relationships (common ones)
    KNOWN_DEPS = {
        'react-dom': ['react'],
        'react-router-dom': ['react', 'react-dom'],
        '@vitejs/plugin-react': ['vite', 'react'],
        'tailwindcss': ['postcss', 'autoprefixer'],
        'express': ['http-errors', 'content-type'],
        'helmet': ['express'],
        'cors': ['express'],
        'compression': ['express'],
    }

    missing = []
    for dep in all_deps:
        name = dep.split('@')[0] if '@' in dep else dep
        if name in KNOWN_DEPS:
            for peer in KNOWN_DEPS[name]:
                G.add_edge(name, peer)
                if peer not in [d.split('@')[0] if '@' in d else d for d in all_deps]:
                    missing.append({'package': peer, 'required_by': name, 'type': 'peer_dependency'})

    # Check for circular dependencies
    cycles = list(nx.simple_cycles(G))

    # Compute importance (PageRank)
    try:
        importance = nx.pagerank(G)
        critical = sorted(importance.items(), key=lambda x: x[1], reverse=True)[:5]
    except:
        critical = []

    elapsed = time.time() - start
    return {
        'total_deps': len(deps),
        'total_dev_deps': len(dev_deps or []),
        'missing_peers': missing,
        'circular_deps': cycles,
        'critical_packages': [{'package': k, 'importance': round(v, 4)} for k, v in critical],
        'graph_nodes': G.number_of_nodes(),
        'graph_edges': G.number_of_edges(),
        'latency_ms': round(elapsed * 1000),
    }

def scaffold_project(description, preferred_template=None):
    """Scaffold a new project from natural language description"""
    start = time.time()

    # Smart template matching via embeddings
    if not preferred_template:
        desc_vec = embedder.encode(description)
        similarities = {k: float(cosine_similarity([desc_vec], [v])[0][0]) for k, v in TEMPLATE_VECTORS.items()}
        preferred_template = max(similarities, key=similarities.get)
        match_confidence = similarities[preferred_template]
    else:
        match_confidence = 1.0

    template = TEMPLATES.get(preferred_template, TEMPLATES['node-api'])

    # Generate project structure with LLM
    prompt = f"""You are BUILDER, a project scaffolding expert. Create a project structure for:
{description}

Template: {preferred_template}
Required files: {json.dumps(template['files'])}
Dependencies: {json.dumps(template['deps'])}

Generate a brief description of what each file should contain and any additional files needed:"""

    messages = [{'role': 'user', 'content': prompt}]
    formatted = generator.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    result = generator(formatted, max_new_tokens=500, temperature=0.4, do_sample=True)
    structure = result[0]['generated_text'][len(formatted):].strip()

    elapsed = time.time() - start
    return {
        'template': preferred_template,
        'match_confidence': round(match_confidence, 4),
        'files': template['files'],
        'dependencies': template['deps'],
        'dev_dependencies': template['dev_deps'],
        'structure_guide': structure,
        'dep_analysis': analyze_dependencies(template['deps'], template['dev_deps']),
        'latency_ms': round(elapsed * 1000),
        'device': device,
    }

def hydrate_project(existing_files, project_type=None):
    """Analyze existing project and suggest missing components"""
    start = time.time()

    # Detect project type from files
    if not project_type:
        if any('package.json' in f for f in existing_files):
            if any('.jsx' in f or '.tsx' in f for f in existing_files):
                project_type = 'react-app'
            else:
                project_type = 'node-api'
        elif any('requirements.txt' in f or 'setup.py' in f for f in existing_files):
            project_type = 'python-api'
        else:
            project_type = 'fullstack'

    template = TEMPLATES.get(project_type, TEMPLATES['node-api'])
    existing_basenames = [f.split('/')[-1] for f in existing_files]

    missing = []
    for template_file in template['files']:
        basename = template_file.split('/')[-1]
        if basename not in existing_basenames:
            missing.append(template_file)

    # Essential files check
    essential_missing = []
    essentials = ['.gitignore', 'README.md', '.env.example']
    for e in essentials:
        if e not in existing_basenames:
            essential_missing.append(e)

    elapsed = time.time() - start
    return {
        'detected_type': project_type,
        'existing_files': len(existing_files),
        'template_files': len(template['files']),
        'missing_files': missing,
        'missing_count': len(missing),
        'essential_missing': essential_missing,
        'completeness': round((1 - len(missing) / max(len(template['files']), 1)) * 100, 1),
        'recommended_deps': template['deps'],
        'latency_ms': round(elapsed * 1000),
    }

def analyze_architecture(files, file_contents=None):
    """Generate architecture analysis from file structure"""
    start = time.time()

    # Categorize files
    categories = {
        'routes': [], 'models': [], 'middleware': [], 'components': [],
        'utils': [], 'config': [], 'tests': [], 'styles': [], 'other': [],
    }

    for f in files:
        lower = f.lower()
        if any(x in lower for x in ['route', 'api', 'endpoint']): categories['routes'].append(f)
        elif any(x in lower for x in ['model', 'schema', 'entity']): categories['models'].append(f)
        elif any(x in lower for x in ['middleware', 'auth', 'guard']): categories['middleware'].append(f)
        elif any(x in lower for x in ['component', '.jsx', '.tsx', '.vue']): categories['components'].append(f)
        elif any(x in lower for x in ['util', 'helper', 'lib']): categories['utils'].append(f)
        elif any(x in lower for x in ['config', '.env', '.yaml', '.yml']): categories['config'].append(f)
        elif any(x in lower for x in ['test', 'spec', '__test__']): categories['tests'].append(f)
        elif any(x in lower for x in ['.css', '.scss', '.less', 'style']): categories['styles'].append(f)
        else: categories['other'].append(f)

    # Compute metrics
    dirs = set()
    extensions = {}
    for f in files:
        parts = f.split('/')
        if len(parts) > 1:
            dirs.add(parts[0])
        ext = f.rsplit('.', 1)[-1] if '.' in f else 'no_ext'
        extensions[ext] = extensions.get(ext, 0) + 1

    elapsed = time.time() - start
    return {
        'total_files': len(files),
        'directories': len(dirs),
        'categories': {k: {'count': len(v), 'files': v} for k, v in categories.items() if v},
        'extensions': dict(sorted(extensions.items(), key=lambda x: x[1], reverse=True)),
        'has_tests': len(categories['tests']) > 0,
        'test_coverage_estimate': round(len(categories['tests']) / max(len(files), 1) * 100, 1),
        'latency_ms': round(elapsed * 1000),
    }

# Quick test
print('=== BUILDER Quick Test ===')
scaffold = scaffold_project('A real-time dashboard for monitoring AI agent performance')
print(f'Template matched: {scaffold["template"]} (confidence: {scaffold["match_confidence"]})')
print(f'Files: {len(scaffold["files"])}, Deps: {len(scaffold["dependencies"])}')

In [ ]:
# Cell 5: FastAPI server
from fastapi import FastAPI, Request
import uvicorn
import threading
import asyncio
import httpx

app = FastAPI(title='BUILDER GPU Node', version='1.0.0')

@app.get('/health')
async def health():
    return {
        'status': 'active',
        'node_id': HEADY_CONFIG['node_id'],
        'node_role': HEADY_CONFIG['node_role'],
        'gpu': torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'cpu',
        'gpu_memory_used_gb': round(torch.cuda.memory_allocated(0) / 1e9, 2) if torch.cuda.is_available() else 0,
        'models': ['all-MiniLM-L6-v2', 'TinyLlama-1.1B-Chat'],
        'capabilities': HEADY_CONFIG['capabilities'],
        'supported_tasks': list(SUPPORTED_TASKS.keys()),
        'templates': list(TEMPLATES.keys()),
    }

@app.post('/api/builder/scaffold')
async def api_scaffold(request: Request):
    data = await request.json()
    return scaffold_project(data.get('description', ''), data.get('template', None))

@app.post('/api/builder/hydrate')
async def api_hydrate(request: Request):
    data = await request.json()
    return hydrate_project(data.get('files', []), data.get('project_type', None))

@app.post('/api/builder/dependencies')
async def api_deps(request: Request):
    data = await request.json()
    return analyze_dependencies(data.get('deps', []), data.get('dev_deps', []))

@app.post('/api/builder/architecture')
async def api_arch(request: Request):
    data = await request.json()
    return analyze_architecture(data.get('files', []))

@app.post('/api/tasks/execute')
async def execute_task(request: Request):
    data = await request.json()
    task_type = data.get('type', 'build_project')
    payload = data.get('payload', data)
    try:
        if task_type in ('build_project', 'dependency_analysis'):
            result = analyze_dependencies(payload.get('deps', []), payload.get('dev_deps', []))
        elif task_type == 'newproject':
            result = scaffold_project(payload.get('description', ''), payload.get('template', None))
        elif task_type == 'hydrate':
            result = hydrate_project(payload.get('files', []), payload.get('project_type', None))
        elif task_type == 'architecture':
            result = analyze_architecture(payload.get('files', []))
        else:
            result = scaffold_project(json.dumps(payload)[:300])
        return {'success': True, 'node_id': HEADY_CONFIG['node_id'], 'task_type': task_type, 'result': result}
    except Exception as e:
        return {'success': False, 'error': str(e), 'node_id': HEADY_CONFIG['node_id']}

print('BUILDER FastAPI ready: /health, /api/builder/scaffold, /api/builder/hydrate, /api/builder/dependencies, /api/builder/architecture, /api/tasks/execute')

In [ ]:
# Cell 6: ngrok tunnel + auto-register with HeadyCloud
from pyngrok import ngrok, conf

# Authenticate ngrok
conf.get_default().auth_token = "39ZBirdUD63xgta7yN7OFZpE84m_3QZyJTDno1b8Yhv9Nfy8s"

public_url = ngrok.connect(HEADY_CONFIG['port']).public_url
print(f'BUILDER GPU Node live at: {public_url}')

PUBLIC_URL = public_url

async def register_with_clouds():
    async with httpx.AsyncClient(timeout=15) as client:
        for endpoint in HEADY_CONFIG['registration_endpoints']:
            try:
                resp = await client.post(endpoint, json={
                    'node_id': HEADY_CONFIG['node_id'],
                    'url': PUBLIC_URL,
                    'role': HEADY_CONFIG['node_role'],
                    'capabilities': HEADY_CONFIG['capabilities'],
                    'gpu': torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'cpu',
                })
                print(f'  Registered with {endpoint}: {resp.status_code}')
            except Exception as e:
                print(f'  Pending: {endpoint} ({e})')

async def heartbeat_loop():
    while True:
        await asyncio.sleep(30)
        async with httpx.AsyncClient(timeout=10) as client:
            for layer, url in HEADY_CONFIG['cloud_layers'].items():
                try:
                    await client.post(f'{url}/api/nodes/heartbeat', json={
                        'node_id': HEADY_CONFIG['node_id'],
                        'status': 'active',
                        'url': PUBLIC_URL,
                        'metrics': {
                            'gpu': torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'cpu',
                            'gpu_memory_used': round(torch.cuda.memory_allocated(0) / 1e9, 2) if torch.cuda.is_available() else 0,
                            'models_loaded': 2,
                        }
                    })
                except:
                    pass

asyncio.run(register_with_clouds())
threading.Thread(target=lambda: asyncio.run(heartbeat_loop()), daemon=True).start()
print(f'BUILDER GPU node live at {PUBLIC_URL} — accepting build requests')
# 0.0.0.0 is the Colab VM bind address for uvicorn, NOT a local service
uvicorn.run(app, host='0.0.0.0', port=HEADY_CONFIG['port'])